In [2]:
import numpy as np
from matplotlib import pyplot as plt
import json
from tqdm import tqdm
import sys
import corner
import matplotlib.image as mpimg

sys.path.append("../")

import tdpy

%load_ext autoreload
%autoreload 2

In [2]:
def gen_arr(config, bin=True, n=12000):
    n_secondary = config["n_secondary"]
    l = bin*len(config["binary"].keys()) + n_secondary*len(config["secondary_0"].keys()) + 1
    arr = np.zeros((n, l))*np.nan
    keys = []
    keys.append("m_star")
    for i in range(0, n_secondary):
        for key in config["secondary_0"].keys():
            keys.append(f"{i}/{key}")
    if bin:
        for key in config["binary"].keys():
            keys.append(f"bin/{key}")
    
    return arr, keys




def add_to_array(arr, config, keys, i, bin=True, n=12000):
    if not np.any(arr):
        arr, keys = gen_arr(config, bin=bin, n=n)
        
    n_secondary = config["n_secondary"]
    arr[i, 0] = config["m_star"]
    j = 1
    
    for k in range(0, n_secondary):
        for key in config["secondary_0"].keys():
            arr[i, j] = config[f"secondary_{k}"][key]
            j += 1
    if bin:
        for key in config["binary"].keys():
            arr[i, j] = config["binary"][key]
            j += 1
    
    return arr, keys
    

In [3]:
dir = "no_inj"
system = "K2-72"

keys = None
arr = None
j = 0
stable = np.zeros(12000)

for i in tqdm(range(1, 12000)):
    try:
        with open(f"../output/{dir}/{system}/{i}/config.json") as f:
            cfg = json.load(f)
            arr, keys = add_to_array(arr, cfg, keys, j, bin=False)
            
        elements = np.load(f"../output/{dir}/{system}/{i}/elements.npy")
        a_i = elements[0, :, 0]
        a_f = elements[-1, :, 0]
        stable[i] = np.all(((a_i-a_f)/a_i) < 0.1)
        j += 1

    except Exception as e:
        # print(e)
        pass
arr = arr[:j, :]
stable = stable[:j]
stable = np.logical_not(np.logical_not(stable))

100%|██████████| 11999/11999 [00:15<00:00, 795.41it/s] 


In [24]:
data = np.load("results.npz", allow_pickle=True)['arr_0'].tolist()

In [28]:
dir = "no_inj"
system = "K2-72"

keys = None
arr = None
j = 0
stable = np.zeros(12000)

for i in tqdm(range(1, 12000)):
    try:
        with open(f"../output/{dir}/{system}/{i}/config.json") as f:
            cfg = json.load(f)
            arr, keys = add_to_array(arr, cfg, keys, j, bin=False)
            
        elements = np.load(f"../output/{dir}/{system}/{i}/elements.npy")
        a_i = data[dir][system]["a_i"][i]
        a_f = data[dir][system]["a_f"][i]
        stable[i] = np.abs(data[dir][system]["norm"][i]) < 0.1
        j += 1

    except Exception as e:
        print(e)
        pass
arr = arr[:j, :]
stable = stable[:j]
stable = np.logical_not(np.logical_not(stable))

  8%|▊         | 1019/11999 [00:00<00:01, 10185.20it/s]

[Errno 2] No such file or directory: '../output/no_inj/K2-72/1/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/2/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/3/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/4/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/5/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/6/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/7/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/8/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/9/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/10/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/12/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/13/config.json'
[Errno 2

 17%|█▋        | 2038/11999 [00:00<00:01, 5306.56it/s] 

[Errno 2] No such file or directory: '../output/no_inj/K2-72/1394/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1395/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1396/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1397/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1398/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1399/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1400/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1401/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1402/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1403/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1404/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/1405/config.json'
[Errno 2] No such file or directory: '../output/no_i

 22%|██▏       | 2692/11999 [00:00<00:02, 3695.11it/s]

...

 93%|█████████▎| 11134/11999 [00:03<00:00, 3963.25it/s]

[Errno 2] No such file or directory: '../output/no_inj/K2-72/11023/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11024/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11025/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11026/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11027/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11028/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11029/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11030/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11031/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11032/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11033/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11034/config.json'
[Errno 2] No such file or directory: '..

100%|██████████| 11999/11999 [00:03<00:00, 3626.75it/s]

[Errno 2] No such file or directory: '../output/no_inj/K2-72/11524/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11525/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11526/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11527/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11528/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11529/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11530/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11531/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11532/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11533/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11534/config.json'
[Errno 2] No such file or directory: '../output/no_inj/K2-72/11535/config.json'
[Errno 2] No such file or directory: '..

TypeError: 'NoneType' object is not subscriptable

In [26]:
data["no_inj"]["Kepler-150"]

{'a_i': array([[0.04041227, 0.06723912, 0.10568356, 0.15946177, 1.03577795],
        [0.04997443, 0.0625469 , 0.12081896, 0.20725737, 1.5245469 ],
        [0.0523605 , 0.07939835, 0.09042068, 0.18832008, 1.50683142],
        ...,
        [0.04083403, 0.07847224, 0.09314797, 0.22187988, 1.44665833],
        [0.03891349, 0.08172847, 0.12042333, 0.1583142 , 1.08038071],
        [0.0433161 , 0.08142082, 0.083691  , 0.21922781, 1.45418252]]),
 'a_f': array([[0.04041003, 0.0672391 , 0.10565291, 0.15947199, 1.03645652],
        [0.04998404, 0.06254422, 0.12083504, 0.20725679, 1.52429184],
        [0.05236311, 0.07939669, 0.09043088, 0.18830977, 1.50700794],
        ...,
        [0.04083432, 0.07847022, 0.09315528, 0.22184337, 1.44646946],
        [0.0389135 , 0.08172323, 0.12039534, 0.15835912, 1.08012975],
        [0.        , 0.        , 0.        , 0.        , 0.        ]]),
 'norm': array([ 2.90035577e-04,  1.67302303e-04,  5.47682904e-05,  1.14876501e-04,
         6.75009996e-04,  2.7911

In [5]:
idxs = np.array(["inc" not in key for key in keys])

param_labels = np.char.replace(np.array(keys)[idxs], "_", "-")



# labels of the populations
listlablpopl = ['stable', 'not stable']

# number of populations
numbpopl = len(listlablpopl)


# number of features
numbfeat = np.sum(idxs)

# generate samples


# labels of the parameters
listlablpara = []
for k in range(numbfeat):
    listlablpara.append([param_labels[k], ''])

typepgrp = 'both'
    
tdpy.plot_grid( [arr[stable, :][: ,idxs], arr[np.logical_not(stable),:][:,idxs]], listlablpara, listlablpopl=listlablpopl, typeplottdim='scat', pathbase="../figs/corner")
# plt.savefig("../figs/corner.pdf")


Number of samples in the populations:
not stable
2
stable
998
Writing to ../figs/cornerpmar_scat_None.png...


In [ ]:
plt.imshow(mpimg.imread('../figs/cornerpmar_scat_None.png'))

In [52]:
arr[stable, :][:, idxs].shape

(14, 31)

In [47]:
.shape

(2000,)